# Deploy trained model to Endpoint

Endpoints provide a unified interface to invoke and manage model deployments across compute types [What are endpoints and deployments?](https://learn.microsoft.com/en-us/azure/machine-learning/concept-endpoints?view=azureml-api-2)


In [1]:
from azure.ai.ml import MLClient
from azure.ai.ml.entities import ManagedOnlineEndpoint, ManagedOnlineDeployment, CodeConfiguration, OnlineRequestSettings
from azure.identity import DefaultAzureCredential

## Define global variables

In [34]:
# Registry
registry_name="ams-components"
registry_location="westeurope"

#Registry Components
registered_model_name = 'SegResNet'

# Deployment
online_endpoint_name = 'SRN-endpoint-model-from-registry'
deployment_name = 'blue'
inference_env_name = 'inference-env'
default_instance_type = "Standard_NC4as_T4_v3"

## Create Azure ML and Registry Clients

In [3]:
credential = DefaultAzureCredential()

ml_client = MLClient.from_config(credential=credential)

ml_client_registry = MLClient(credential=credential,
                        registry_name=registry_name,
                        registry_location=registry_location)

Found the config file in: ./config.json


## Get latest model and environment from Registry

In [16]:
model = ml_client_registry.models.get(name=registered_model_name, version=str(1))
scoring_environment = ml_client_registry.environments.get(name=inference_env_name, version=str(1))

## Create Endpoint

In [17]:
endpoint = ManagedOnlineEndpoint(
    name=online_endpoint_name,
    description="An online endpoint to generate segmentation predictions",
    auth_mode="key",
    tags={"model": "segreanet"},
)

ml_client.begin_create_or_update(endpoint)

## Deploy deployment is a set of resources required for hosting the model
_Wait until Endpoint is created_

In [35]:
#latest_env_name = inference_env_name + ':1' 
blue_deployment = ManagedOnlineDeployment(
    name=deployment_name,
    endpoint_name=online_endpoint_name,
    model=model,
    environment=scoring_environment,
    code_configuration=CodeConfiguration(
        code="./",
        scoring_script="score.py"
    ),
    instance_type=default_instance_type,
    instance_count=3,
    request_settings= OnlineRequestSettings(request_timeout_ms = 90000),
)

deployment = ml_client.begin_create_or_update(blue_deployment)

Check: endpoint SRN-endpoint-model-from-registry exists
Uploading notebooks (0.53 MBs): 100%|██████████| 530752/530752 [00:00<00:00, 8159871.11it/s]


data_collector is not a known attribute of class <class 'azure.ai.ml._restclient.v2022_02_01_preview.models._models_py3.ManagedOnlineDeployment'> and will be ignored


..............................................

## Set Endpoint traffic

In [ ]:
# blue deployment takes 100 traffic
endpoint.traffic = {deployment_name: 100}
ml_client.online_endpoints.begin_create_or_update(endpoint).result()